# <img style="float: left; padding-right: 10px; width: 45px" src="https://raw.githubusercontent.com/Harvard-IACS/2018-CS109A/master/content/styles/iacs.png"> CS207 Systems Development for Computational Science
## Final Project
## Future Features



**Harvard University**<br/>
**Fall 2019**<br/>
**Instructor**: David Sondak <br/>
**Team #14**: Fantastic Four<br/>
**Students**: Daniel Cox, Anna Davydova, Stephen Moon, Valentina Toll Villagra   <br/>
**Git Repository**:https://github.com/IACS-CS-207-FantasticFour/cs207-FinalProject


<hr style="height:2pt">

*“Nothing takes place in the world whose meaning is not that of some maximum or minimum.”*
― Leonhard Euler

**Future Features**

In the financial markets, large institutions (e.g. banks, asset managers) hold complex portfolios with a variety of financial instruments. While calculating the value of such portfolios can be done fairly easily with simulations and aggregation of market pricing, the sensitivities of these portfolios to changes in price, volatility and interest rates in the market can be a lot more difficult to calculate and come with an accuracy vs. efficiency trade off. 


Given the time and the scope of this project we chose to focus on a subset of the financial instruments and potential end users.  Specifically, we aim to extend our DeltaPi package into a second package HedgeDeltaPi that would be useful to equity options traders as they estimate sensitivities  of their options portfolios and purchase underlying assets (delta-hedge) to minimize their volatility. Accurate and efficient estimate of these sensitivities would allow options traders to buy the right amounts of options to hedge their stock positions from big swings in the asset prices (more on this later).

Here we will focus on vanilla, European, non-dividend paying stock options that can only be exercised on specified date of their expiration as opposed to American options that can be exercised during any time up until the expiration date. 

***What are Financial Options?***

A call option is a right but not an obligation to buy an underlying asset (in our case stock) at a given price (strike price) on a given date (expiration date).  Note from the plot below that the option has no value below the strike price of USD100.  However, as the stock reaches that value and climbs higher, the value of the option is the difference between $100 and the current price (in the >100 region). That makes sense because we can buy the stock at a 100 and sell it at a price that is now >100, netting a profit that is now the value of the option.

$$\text{Value of European Call Option}$$

![](http://www.quantopia.net/wp-content/uploads/2013/03/EuroCallRates.png)</br>
$$\text{source: www.quantopia.net/wp-content/uploads/2013/03/EuroCallRates.png}$$

A put option is a right but not an obligation to sell an underlying asset (in our case stock) at a given price (strike) on a given date (expiration).  The plot below shows that the put option has no value until the stock price drops below the strike. Then the value of the option is the profit one would net by buying the stock <100 and selling it at a strike price of USD100. 

$$\text{Value of European Put Option}$$

![](http://www.quantopia.net/wp-content/uploads/2013/03/EuroPutRates.png)<br/>
$$\text{source: www.quantopia.net/wp-content/uploads/2013/03/EuroPutRates.png}$$

The classical approach to valuing options is with Black-Scholes Equation that describes the option price overtime as follows:

$$\frac{\partial V}{\partial t}+\frac{1}{2}\sigma^2 S^2\frac{\partial^2 V}{\partial S^2}+r S\frac{\partial V}{\partial S} -r V = 0$$

where $V$ is the value of the option, $t$ is time, $\sigma$ is volatility of the underlying asset, $S$ is the price of the underlying asset and $r$ is the risk free rate (Black and Scholes, 1973).

Further, Black-Sholes-Merton formula for estimating price of the European Call option is as follows:

$$C(S_0,t)=S_0N(d_1)-Ke^{-r(T-t)}N(d_2)$$

and similarly the put option value is as follows:


$$P(S_0,t)=Ke^{-r(T-t)}N(-d_2)-S_tN(-d_1)$$

where $S_0$ is the stock price, $C(S_0,t)$ is a price of the call option and $P(S_0,t)$ is price of a put option as a function of stock price and time, $K$ is the strike prices, $(T-t)$ is time to maturity represented in years and $N(d_1)$ and $N(d_2)$ are  cumulative distribution functions for a standard normal distribution (Merton, 1973).

$$d_1=\frac{ln(\frac{S_t}{K})+(r+\frac{\sigma^2_s}{2})(T-t)}{\sigma_s\sqrt{(T-t)}}$$

$$d_2=\frac{ln(\frac{S_t}{K})+(r-\frac{\sigma^2_s}{2})(T-t)}{\sigma_s\sqrt{(T-t)}}=d_1-\sigma_s\sqrt{(T-t)}$$

$$N(d_1)=\int_{-\infty} ^{d_1}f(u)du=\int_{-\infty}^{d_1}\frac{1}{\sqrt{2\pi}}e^{\frac{-u^2}{2}}du$$

$$\dot{N(d_1)}=\frac{\partial N(d_1)}{\partial d_1}=\frac{1}{\sqrt{2\pi}}e^{-\frac{d_1^2}{2}}$$

We can solve for both put and call deltas and obtain a closed form solution in the form. 

$$\Delta_{call}=\frac{\partial C_t}{\partial S_t}=N(d_1)+S_t\frac{\partial N(d_1)}{\partial S_t}-Ke^{-r(T-t)}\frac{\partial N(d_2)}{\partial S_t}= N(d_1)$$

$$\Delta_{put}=\frac{\partial P_t}{\partial S_t}=Ke^{-r(T-t)}\frac{\partial N(-d_2)}{\partial S_t}-N(-d_1)-S_t\frac {\partial N(-d_1)}{\partial S_t}=N(d_1)-1$$

We can also obtain closed form solution for first order partial derivative w.r.t. volatility - Vega (note its the same for both put and call options):

$$v=\frac{\partial{C_t}}{\partial\sigma}=\frac{\partial{P_t}}{\partial\sigma}=S_t\sqrt{T} N(d_1)$$

However, it is important to keep in mind that the $\sigma$ (volatility) of the option is not static overtime (i.e. the volatility surface is not flat).  Thus, rather then focusing on simple delta above, our package will take this into account and calculate minimum variance delta that minimizes volatility of a delta neutral portfolio (i.e. more accurate than static delta).  For local volatility models, several papers ( Derman et al (1995), and Coleman et al (2001)) have shown that:

$$\Delta_{MV}=\Delta_{BS}+v_{BS}\frac{\partial{\sigma_{imp}}}{\partial{K}}$$

where $\Delta_{BS}$ $v_{BS}$ are Black Scholes delta and vega respectively, and $\sigma_{imp}$ is the value of implied volatility. Implied volatility is estimated by the following equation (Corrado et. al., 1996):

$$\sigma_{imp}\approx\sqrt{\frac{2\pi}{(T-t)}} \frac{1}{S+K}(C-\frac{S-K}{2}+\sqrt{(C-\frac{S-K}{2})^2-\frac{(S-K)^2}{\pi}})$$

DeltaHedgePi will leverage DeltaPi package to calculate the first derivative of $\sigma_{imp}$ and will calculate $\Delta_{BS}$ and $v_{BS}$ using market data inputed by the traders.  Thus, a more realistic/accurate delta will be provided to the end-user for delta hedging purposes.

***What is Delta Hedging?***

In the simple case of vanilla, non dividend paying, European call/put options above, delta hedging refers to buying or selling specific amounts of underlying stock to neutralize the impact of the stock price moves on the options portfolio.  Let us consider the following example:

Consider the following example. Let's supposed we have a stock XYZ that currently has a price of USD100 and the bank sold 10 call options (a while ago before the stock price run up) for 1000 shares with strike USD10.  Thus, the buyer of our option has the right but not obligation to buy 1000 shares of XYZ at just USD10 on expiration date (right now the option is deep in the money). If the delta of this call option is 0.5, this means that for every USD1 increase in stock price, the value of the call option goes up USD0.50 (to the holder).  To execute a delta hedge, the bank should buy 1000*.5=500 shares of XYZ. Now if the stock price goes down by USD1, the bank will lose USD500 on the stock position but will gain 0.5x1000=USD500 on its short call option position. Thus, the impact is neutral.  

Keep in mind that delta is not static as the slope of option price vs. stock price is usually non linear. This is why accurate and frequent estimates of delta are so important (Lee, 2008).

HedgeDeltaPi will apply automatic differentiation from DeltaPi to compute deltas for call and put options above. We will start by applying the forward mode and will also explore reverse mode AD as it is more efficient for scaling the packages to more complex portfolios (Homescu, 2011). 

**REFERENCES:**<br/>
Atilim Baydin. Automatic Differentiation in Machine Learning: a Survey. Journal of Machine Learning Research, 2018. <br/> 
Fischer Black, & Myron Scholes, M. The Pricing of Options and Corporate Liabilities. Retrieved November 1, 2019, from http://www.jstor.org/stable/1831029<br/>
Cheng-Few Lee, 'Handbook on Quantitative Finance and Risk Management.' Rutgers University. 2008.  
George F. Corliss. Application of differentiation arithmetic, volume 19 of Perspectives in Computing, pages 127–48. Academic Press, Boston, 1988.<br/>
Andreas Griewank and Andrea Walther. Evaluating Derivatives: Principles and Techniques of Algorithmic Differentiation. Humboldt-Universität zu Berlin, Berlin, Germany. Second Edition, 2008.<br/>
Michael Heath. Scientific Computing: An Introductory Survey. Society for Industrial and Applied Mathematics. 8(6):367, 2018.<br/> 
Christina Homescu. Adjoints and automatic (algorithmic) differentiation in computational finance. arXiv:1107.1831v1 10 Jul 2011.<br/>
Max E. Jerrell. Automatic differentiation and interval arithmetic for estimation of disequilibrium models. Computational Economics, 10(3):295–316, 1997.<br/>
Merton, R. Theory of Rational Option Pricing. Retrieved November 1, 2019, from http://www.jstor.org/stable/3003143 <br/>
David Sondak. Lectures 10 - 11. Harvard University. CS207 Fall 2019. <br/> 
Eric Sunnegardh and Ludvig Lamm. "Efficient Sensitivity Analysis Using Algorithmic Diferentation in Financial Applications". 2015. 
